# NEI Simulation Training Notebook

This notebook provides a step-by-step guide on how to configure, run, and visualize Non-Equilibrium Ionization (NEI) simulations using the `py_nei` package.

This material is intended for training purposes.

## 1. Setup Environment and Imports

First, we need to import the necessary modules `py_nei.nei_simulation`.

In [1]:
import os
import sys
from datetime import datetime

import h5py
import numpy as np

import matplotlib.pyplot as plt

# Add the current directory to sys.path to ensure we can import the py_nei module
# Adjust this if your notebook is moved to a subfolder
current_dir = os.path.abspath('.')
if current_dir not in sys.path:
    sys.path.append(current_dir)

from py_nei.nei_simulation import NEISimulation

## 2. Configuration

Define the paths for the executable and the working directory. 
We use a timestamp to create a unique folder for each run, preventing data overwrite.

In [2]:
# Path to the compiled executable (nei_exp3.exe)
exe_fname = os.path.join(current_dir, "bin", "nei_exp4.exe") 

# Generate a unique working directory name using the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
working_directory = os.path.join(current_dir, "runs", f"notebook_case_{timestamp}")

# Eigentable_directory: including both the Maxwellian and Kappa eigentables
eigentable_directory = os.path.join("/Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/One_kappa_table/create_eigentables/kappa_tables/")
print(f"Executable file: {exe_fname}")
print(f"Working Directory: {working_directory}")
print(f"Eigentable Directory: {eigentable_directory}")

Executable file: /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/bin/nei_exp4.exe
Working Directory: /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/runs/notebook_case_20260201_122637
Eigentable Directory: /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/One_kappa_table/create_eigentables/kappa_tables/


## 3. Initialize Simulation

Create an instance of the `NEISimulation` class. This wrapper manages the entire workflow.

In [3]:
# 3.1 Initialize without arguments
simulation = NEISimulation(exe_fname=exe_fname,
                            working_dir=working_directory,
                            eigentable_dir=eigentable_directory)

# 3.3 Configure the input data (time, temperature, density, you can load from file or manually input here)
# Load the temperature & density evolution history data from 
# /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/te_ne_history.h5
input_h5file = "/Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/te_ne_history.h5"
with h5py.File(input_h5file, 'r') as h5f:
    # Read datasets into 1D numpy arrays 
    time_data = h5f['time'][:]
    temp_data = h5f['temperature'][:]
    dens_data = h5f['density'][:]

# Apply data to the simulation as the plamsa history
simulation.set_history_data(
    time = time_data,
    temperature = temp_data,
    density = dens_data
)


2026-02-01 12:26:37,276 - py_nei.nei_simulation - WARNING - Working directory /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/runs/notebook_case_20260201_122637 does not exist. Creating it.
2026-02-01 12:26:37,279 - py_nei.nei_simulation - INFO - Parsed kappa values: [  1.7     1.702   1.703   1.705   1.706   1.708   1.71    1.711   1.713
   1.714   1.716   1.718   1.72    1.721   1.723   1.725   1.726   1.728
   1.73    1.732   1.734   1.735   1.737   1.739   1.741   1.743   1.745
   1.747   1.749   1.75    1.752   1.754   1.756   1.758   1.76    1.762
   1.764   1.767   1.769   1.771   1.773   1.775   1.777   1.779   1.781
   1.784   1.786   1.788   1.79    1.793   1.795   1.797   1.799   1.802
   1.804   1.806   1.809   1.811   1.814   1.816   1.819   1.821   1.824
   1.826   1.829   1.831   1.834   1.836   1.839   1.842   1.844   1.847
   1.85    1.852   1.855   1.858   1.861   1.863   1.866   1.869   1.872
   1.875   1.878   1.881   1.884   1.887   1.89    1.893   1

## 4. Set Parameters

Here we define the key simulation parameters.
- `ws_path_1`: Path to the ionization and recombination tables. 
               You may employ different rate coefficient by loading different tables.
- `ws_path_3`: Path to initial ionization states.
- `ws_path_4`: Path for output files (usually the working directory itself).
- `kappa`: The kappa distribution index. Default is 0.0 (Maxwellian).

**Note**: Path strings in the parameters dictionary usually require inner single quotes (e.g., `"'path'"`).

In [4]:
# Define simulation parameters
params = {
    # Note: Paths usually need to be wrapped in single quotes for the input file format
    "ws_path_1": "'/Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/One_kappa_table/create_eigentables/kappa_tables/'", 
    "ws_path_2": f"'{working_directory}/'", # Can be empty as it has been set in the above initialization
    "ws_path_3": "'/Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/Solarwind_onestreamline/run_NEI_Maxwellian/'",
    # Set output path to our current working directory
    "ws_path_4": f"'{working_directory}/'",
    "output_basename": "'run_notebook_'",
    # Set Kappa Parameter (Optional)
    #kappa_values": 6.0 # Set any value
    "kappa_values": 'Maxwell' # 'Maxwell'
    #"kappa_values": np.linspace(6.0, 5.0, num=len(time_data)) # Kappa values changing over time
                                        # It should match the time range of the history data
}

# Apply parameters
simulation.set_parameters(params)


## 5. Generate Input File, History Data, and Run the simulation

- Generate the `input.txt` file required by the Fortran executable.</br>
- Generate the `te_ne_history.dat` file required by the Fortran executable.

In [5]:
# if kappa is single value then generate input file directly for the whole 
# history, else loop over kappa values to generate multiple input files
kappa_values = simulation.parameters.get('kappa_values')
print(kappa_values)
print(time_data)

simulation.generate_history_files()


2026-02-01 12:26:37,289 - py_nei.nei_simulation - INFO - Input file generated at: /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/runs/notebook_case_20260201_122637/input_00000.txt


Maxwell
[0.00000000e+00 2.78046399e+00 5.60086393e+00 8.46340299e+00
 1.13702953e+01 1.43233864e+01 1.73244526e+01 2.03750228e+01
 2.34766379e+01 2.66310056e+01 2.98398387e+01 3.31047880e+01
 3.64269899e+01 3.98079779e+01 4.32490267e+01 4.67517572e+01
 5.03181035e+01 5.39502597e+01 5.76504638e+01 6.14208074e+01
 6.52630638e+01 6.91792335e+01 7.31714203e+01 7.72417846e+01
 8.13928922e+01 8.56274931e+01 8.99481313e+01 9.43570763e+01
 9.88567181e+01 1.03449806e+02 1.08139749e+02 1.12929483e+02
 1.17821115e+02 1.22816377e+02 1.27916453e+02 1.33122266e+02
 1.38436102e+02 1.43862171e+02 1.49403741e+02 1.55061551e+02
 1.60837589e+02 1.66735362e+02 1.72756924e+02 1.78905038e+02
 1.85185342e+02 1.91600877e+02 1.98150371e+02 2.04836255e+02
 2.11664005e+02 2.18636529e+02 2.25752269e+02 2.33012082e+02
 2.40423743e+02 2.47994962e+02 2.55726652e+02 2.63613640e+02
 2.71654984e+02 2.79856148e+02 2.88226358e+02 2.96769383e+02
 3.05482936e+02 3.14364100e+02 3.23415498e+02 3.32646178e+02
 3.42060858e+02 

## 6. Run Experiment

Execute the simulation. 

In [6]:
simulation.run_simulation()

Running simulation for patch index: 0


2026-02-01 12:26:37,861 - py_nei.nei_simulation - INFO - Experiment finished successfully.


Simulation run completed.


## 7. Visualize Results

Plot the resulting ionization fractions.

In [7]:
# This reads the binary log file specified by 'output_basename' + .log
#logfile = simulation.working_dir + '/run_notebook_demo.log'
[time, conce] = simulation.visualize_evolution()

2026-02-01 12:26:37,865 - py_nei.nei_simulation - INFO - Searching for logfiles in working directory: /Users/chengcaishen/Workspace/project/2025_kappa_py_ATI/py_nei/runs/notebook_case_20260201_122637


time_all shape: (406,)
conce_all shape: (30, 30, 406)
